In [1]:
#%run ./lib.py

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType, DecimalType, TimestampType, DataType, IntegerType
from pyspark.sql.functions import col, lit, trim, substring, concat, udf, upper, initcap
from datetime import datetime
import os

In [3]:
conf = SparkConf() \
    .setAppName("projeto_pbi") \
    .setSparkHome('./spark/home')

LAKE_HOME = os.getenv("LAKE_HOME", "/spark/home")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

22/11/30 01:51:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/30 01:51:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/30 01:51:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/30 01:51:07 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/30 01:51:07 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
TABLE_NAME = 'fact_order_item'

def create_sk(spark, df, key_column_name, table_name):
    sk = {}
    sk = df.select(col(key_column_name).alias("key")).rdd.zipWithIndex()
    new_sk = sk.map(lambda row: list(row[0]) + [row[1] + 1])
    new_sk_map = new_sk.collectAsMap()

    sk_schema = \
        StructType(
            [StructField('key', StringType(), True),
            StructField('SK', LongType(), True)]
        )

    sk_frame = spark.createDataFrame(new_sk, sk_schema)
    sk_frame.write.mode('overwrite').csv('{}/dataset/e-commerce/02_surrogate_key/sk_{}.csv'.format(LAKE_HOME, table_name), header=True)

    return new_sk_map

def locate_sk(mapping: dict):
    return udf(lambda x: mapping.get(x), IntegerType())


def map_sk(spark, table_name):
    sk_table = spark.read.csv('{}/dataset/e-commerce/02_surrogate_key/sk_{}.csv'.format(LAKE_HOME, table_name), header=True)
    dict_sk = sk_table.rdd.map(lambda x: (x[0], int(x[1]))).collectAsMap()
    return dict_sk

In [5]:
df = spark.read.csv('{}/dataset/e-commerce/01_extract/order_items_dataset.csv'.format(LAKE_HOME), header=True)

df_item = \
    df.select(
        col('order_id').cast(StringType()).alias('ID_ORDER'),
        col('order_item_id').cast(StringType()).alias('NUM_ORDER_ITEM'),
        col('product_id').cast(StringType()).alias('ID_PRODUCT'),
        col('seller_id').cast(StringType()).alias('ID_SELLER'),
        col('shipping_limit_date').cast(TimestampType()).alias('DAT_SHIPPING_LIMIT'),
        col('price').cast(DecimalType(15,2)).alias('VAL_PRICE'),
        col('freight_value').cast(DecimalType(15,2)).alias('VAL_FREIGHT'),
    )

order_sk = map_sk(spark=spark, table_name='fact_order')
product_sk = map_sk(spark=spark, table_name='dim_product')
seller_sk = map_sk(spark=spark, table_name='dim_seller')

In [6]:
fact_order_item = \
    df_item \
        .withColumn('SK_ORDER', locate_sk(order_sk)(col('ID_ORDER'))) \
        .withColumn('SK_PRODUCT', locate_sk(product_sk)(col('ID_PRODUCT'))) \
        .withColumn('SK_SELLER', locate_sk(seller_sk)(col('ID_SELLER')))

In [7]:
fact_order_item = \
    fact_order_item.select(
            col('SK_ORDER'),
            col('SK_PRODUCT'),
            col('SK_SELLER'),
            col('NUM_ORDER_ITEM'),
            col('DAT_SHIPPING_LIMIT'),
            col('VAL_PRICE'),
            col('VAL_FREIGHT'),  
        )

fact_order_item.write.mode('overwrite').csv('{}/dataset/e-commerce/04_fact/{}.csv'.format(LAKE_HOME, TABLE_NAME), header=True)

In [8]:
fact_order_item.show()

+--------+----------+---------+--------------+-------------------+---------+-----------+
|SK_ORDER|SK_PRODUCT|SK_SELLER|NUM_ORDER_ITEM| DAT_SHIPPING_LIMIT|VAL_PRICE|VAL_FREIGHT|
+--------+----------+---------+--------------+-------------------+---------+-----------+
|   85268|     25866|      514|             1|2017-09-19 09:45:35|    58.90|      13.29|
|   71854|     27231|      472|             1|2017-05-03 11:05:13|   239.90|      19.93|
|    6299|     22625|     1825|             1|2018-01-18 14:48:30|   199.00|      17.87|
|   22551|     15404|     2024|             1|2018-08-15 10:10:18|    12.99|      12.79|
|    5248|      8863|     1598|             1|2017-02-13 13:57:51|   199.90|      18.14|
|   69450|      3940|      660|             1|2017-05-23 03:55:27|    21.90|      12.69|
|   98567|     22293|     2974|             1|2017-12-14 12:10:31|    19.90|      11.85|
|   47317|      6975|      692|             1|2018-07-10 12:30:45|   810.00|      70.75|
|   92214|      2714|

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
